In [15]:
import requests
import pydicom
import json
from pathlib import Path
from urllib3.filepost import encode_multipart_formdata, choose_boundary
from azure.identity import DefaultAzureCredential
from typing import List

# Authentication
Set up common variables

In [3]:
dicom_service_name = "dicomservice-eastus"
path_to_dicoms_dir = "./dicoms/"

# West US
# base_url = f"https://medicalimaging-ivan-images.dicom.azurehealthcareapis.com/v1" 

# East US
base_url = f"https://healthserviceseastusws-dicomservice-eastus.dicom.azurehealthcareapis.com/v1"

study_uid = "1.2.826.0.1.3680043.8.498.13230779778012324449356534479549187420"; #StudyInstanceUID for all 3 examples
series_uid = "1.2.826.0.1.3680043.8.498.45787841905473114233124723359129632652"; #SeriesInstanceUID for green-square and red-triangle
instance_uid = "1.2.826.0.1.3680043.8.498.47359123102728459884412887463296905395"; #SOPInstanceUID for red-triangle

In [119]:
from azure.identity import DefaultAzureCredential
credential = DefaultAzureCredential()

print(credential.credentials) # this can be used to find the index of the AzureCliCredential

(<azure.identity._credentials.environment.EnvironmentCredential object at 0x00000189F21DF550>, <azure.identity._credentials.managed_identity.ManagedIdentityCredential object at 0x00000189F21DFBB0>, <azure.identity._credentials.shared_cache.SharedTokenCacheCredential object at 0x00000189F21DD8A0>, <azure.identity._credentials.azure_cli.AzureCliCredential object at 0x00000189F21DFDC0>, <azure.identity._credentials.azure_powershell.AzurePowerShellCredential object at 0x00000189F21DFA00>)


In [120]:
# need AzureCliCredential object here
token = credential.credentials[3].get_token('https://dicom.healthcareapis.azure.com')
bearer_token = f'Bearer {token.token}'
print(token.token)

eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyIsImtpZCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyJ9.eyJhdWQiOiJodHRwczovL2RpY29tLmhlYWx0aGNhcmVhcGlzLmF6dXJlLmNvbSIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0L2E0NjJmNDQ3LTc3Y2YtNDFiYy1hOTQ5LWQzNmI4NTg4NjM1MC8iLCJpYXQiOjE2NzcyMjA4NjIsIm5iZiI6MTY3NzIyMDg2MiwiZXhwIjoxNjc3MjI2MDQ5LCJhY3IiOiIxIiwiYWlvIjoiQVZRQXEvOFRBQUFBT2JycmExdEUxNnhEV2JMNkZ2V3ppWG1VUTJqdEdlaEpLcVNHSWlCNlcxNVZHVlRMQmphenllSE5RRmtzWHhBVzUwWWpxWnlCc25sbEtBZFFtcjk4dW91Sk1aaFg0MUNQcGpHS2htVks1amc9IiwiYWx0c2VjaWQiOiIxOmxpdmUuY29tOjAwMDM0MDAxOEFGMDIxOTYiLCJhbXIiOlsicHdkIl0sImFwcGlkIjoiMDRiMDc3OTUtOGRkYi00NjFhLWJiZWUtMDJmOWUxYmY3YjQ2IiwiYXBwaWRhY3IiOiIwIiwiZW1haWwiOiJ0YXJhcG92QGdtYWlsLmNvbSIsImZhbWlseV9uYW1lIjoiVGFyYXBvdiIsImdpdmVuX25hbWUiOiJJdmFuIiwiaWRwIjoibGl2ZS5jb20iLCJpcGFkZHIiOiI3MS4yMzEuMTc3LjUyIiwibmFtZSI6Ikl2YW4gVGFyYXBvdiIsIm9pZCI6Ijg4OGQ4YWU5LWI4MzYtNDgzMy04NmNmLTYyMWNlOTRmYjEwZiIsInB1aWQiOiIxMDAzMjAwMEM4RDNEQzJFIiwicmgiOiIwLkFWa0FSX1JpcE0

In [132]:
# Supporting method for multipart requests
def encode_multipart_related(fields, boundary=None):
    if boundary is None:
        boundary = choose_boundary()

    body, _ = encode_multipart_formdata(fields, boundary)
    content_type = str('multipart/related; boundary=%s' % boundary)

    return body, content_type

# Check if file is a DICOM file
def is_dicom_file(filename):
    try:
        dataset = pydicom.filereader.dcmread(filename, stop_before_pixels=True)
        return True
    except pydicom.errors.InvalidDicomError:
        return False

In [125]:
client = requests.session()


In [126]:
headers = {"Authorization":bearer_token}
url= f'{base_url}/changefeed'
response = client.get(url,headers=headers)

print(response.content)

b'[{"sequence":1,"partitionName":"Microsoft.Default","studyInstanceUid":"1.2.826.0.1.3680043.8.498.13230779778012324449356534479549187420","seriesInstanceUid":"1.2.826.0.1.3680043.8.498.45787841905473114233124723359129632652","sopInstanceUid":"1.2.826.0.1.3680043.8.498.47359123102728459884412887463296905395","action":"Create","timestamp":"2023-02-21T07:55:28.2364373+00:00","state":"Current","metadata":{"00080005":{"vr":"CS","Value":["ISO_IR 192"]},"00080008":{"vr":"CS","Value":["DERIVED","SECONDARY","OTHER"]},"00080016":{"vr":"UI","Value":["1.2.840.10008.5.1.4.1.1.7"]},"00080018":{"vr":"UI","Value":["1.2.826.0.1.3680043.8.498.47359123102728459884412887463296905395"]},"00080020":{"vr":"DA","Value":["20200922"]},"00080023":{"vr":"DA"},"0008002A":{"vr":"DT"},"00080030":{"vr":"TM","Value":["120000"]},"00080033":{"vr":"TM"},"00080050":{"vr":"SH"},"00080060":{"vr":"CS","Value":["OT"]},"00080064":{"vr":"CS","Value":["SYN"]},"00080090":{"vr":"PN","Value":[{"Alphabetic":"Doctor Doc"}]},"0010001

# Sending

In [135]:
def send_dicom_files(path_to_dicom_dir: str, base_url: str, bearer_token: str, batch_size: int = 0):

    file_extension = ""

    # Get a list of all files in the folder with the specified file extension
    folder_path = Path(path_to_dicom_dir)
    file_names = [str(file_path) for file_path in folder_path.rglob(f'*{file_extension}') if Path.is_file(file_path)]

    print(f"Found {len(file_names)} files")

     # Create a list to store the file bytes
    rawfiles = []
    
    # Read the contents of each file and add them to the list
    for file_name in file_names:
        filepath = Path(path_to_dicom_dir).joinpath(file_name)
        # print(f"Loading {filepath}")
        if (is_dicom_file(file_name)):
            with open(filepath, 'rb') as reader:
                rawfile = reader.read()
            rawfiles.append(rawfile)
        else:
            print(f"Not a valid DICOM file, skipping: {file_name}")
    
    print(f"Loaded {len(rawfiles)} files")

    # Create a list of tuples containing the file data and MIME type
    files = []
    for i in range(len(rawfiles)):
        file_data = rawfiles[i]
        file_name = f'dicomfile{i}'
        file_mime_type = 'application/dicom'
        files.append((file_name,('dicomfile', file_data, file_mime_type)))
    
    file_batches = [files]

    if (batch_size > 0):
        # Split the list of files into batches
        file_batches = [files[i:i+batch_size] for i in range(0, len(files), batch_size)]

    responses = []
    print(f"Preparing to send {len(file_batches)} batch{'' if len(file_batches)%10 == 1 else 'es'} of {batch_size} files each")
    for index, file_batch in enumerate(file_batches):
        # Encode the file data as multipart_related
        body, content_type = encode_multipart_related(fields=dict(files))

        # Set the request headers
        headers = {
            'Accept': 'application/dicom+json',
            'Content-Type': content_type,
            'Authorization': bearer_token
        }

        # Send the POST request to the API endpoint
        url = f'{base_url}/studies'
        response = client.post(url, data=body, headers=headers, verify=False)

        responses.append(response)
        print(f"Sent batch {index} of {len(file_batches)}. Status: {response.status_code}")

    print(f"Sent {len(files)} files")

    # Return the response from the API endpoint
    return responses


In [136]:
response = send_dicom_files(r"C:\temp\imaging\jan15\DICOM\DICOM\0000788D\AAEBB2F4", base_url, bearer_token)

Found 403 files


KeyboardInterrupt: 

In [130]:
print(f"Status: {response.status_code}\n")
print(response.content)


Status: 202

b'{"00081198":{"vr":"SQ","Value":[{"00081150":{"vr":"UI","Value":["1.2.840.10008.5.1.4.1.1.4"]},"00081155":{"vr":"UI","Value":["1.3.12.2.1107.5.2.33.37105.2015011615583089226127926"]},"00081197":{"vr":"US","Value":[45070]}},{"00081150":{"vr":"UI","Value":["1.2.840.10008.5.1.4.1.1.4"]},"00081155":{"vr":"UI","Value":["1.3.12.2.1107.5.2.33.37105.2015011615583096827027929"]},"00081197":{"vr":"US","Value":[45070]}},{"00081150":{"vr":"UI","Value":["1.2.840.10008.5.1.4.1.1.4"]},"00081155":{"vr":"UI","Value":["1.3.12.2.1107.5.2.33.37105.2015011615583122750227944"]},"00081197":{"vr":"US","Value":[45070]}},{"00081150":{"vr":"UI","Value":["1.2.840.10008.5.1.4.1.1.4"]},"00081155":{"vr":"UI","Value":["1.3.12.2.1107.5.2.33.37105.2015011615583076829427916"]},"00081197":{"vr":"US","Value":[45070]}},{"00081150":{"vr":"UI","Value":["1.2.840.10008.5.1.4.1.1.4"]},"00081155":{"vr":"UI","Value":["1.3.12.2.1107.5.2.33.37105.2015011615583119266927941"]},"00081197":{"vr":"US","Value":[45070]}},{"0

## Analyzing response

In [131]:
jsondata = json.loads(response.content)
print(f"Response classes: {len(jsondata)} ({[x for x in jsondata]})\n")

failure_tag = "00081198"
success_tag = "00081199"

failure_reasons = {
    "272": "The store transaction didn't store the instance because of a general failure in processing the operation.",
    "43264": "The DICOM instance failed the validation.",
    "43265": "The provided instance StudyInstanceUID didn't match the specified StudyInstanceUID in the store request.",
    "45070": "A DICOM instance with the same StudyInstanceUID, SeriesInstanceUID, and SopInstanceUID has already been stored. If you wish to update the contents, delete this instance first.",
    "45071": "A DICOM instance is being created by another process, or the previous attempt to create has failed and the cleanup process hasn't had chance to clean up yet. Delete the instance first before attempting to create again."
}

if (failure_tag in jsondata):
    print(f"Failures ({failure_tag}):")
    # Parse the JSON text into a Python dictionary
    data = json.loads(response.content)[failure_tag]["Value"]

    # Count the number of objects
    count = len(data)

    # Print DICOM Error Comment for each object
    error_comments = []
    for index, item in enumerate(data):
        print(f"\nObj {index}, Instance UID {item['00081155']['Value'] if '00081155' in item else '[no iud]'}")
        failure_reason = str(item["00081197"]["Value"][0])
        print(f"Failure reason: {failure_reasons[failure_reason] if failure_reason in failure_reasons else f'Unknown ({failure_reason})'}")
        if '00741048' in item:
            for sub_item in item['00741048']['Value']:
                if '00000902' in sub_item:
                    print(sub_item['00000902']['Value'])

    print(f"\nFailed objects: {count}\n-------")

if (success_tag in jsondata):
    print(f"\nSuccesses ({success_tag}): {len(jsondata[success_tag]['Value'])}")


Response classes: 2 (['00081198', '00081199'])

Failures (00081198):

Obj 0, Instance UID ['1.3.12.2.1107.5.2.33.37105.2015011615583089226127926']
Failure reason: A DICOM instance with the same StudyInstanceUID, SeriesInstanceUID, and SopInstanceUID has already been stored. If you wish to update the contents, delete this instance first.

Obj 1, Instance UID ['1.3.12.2.1107.5.2.33.37105.2015011615583096827027929']
Failure reason: A DICOM instance with the same StudyInstanceUID, SeriesInstanceUID, and SopInstanceUID has already been stored. If you wish to update the contents, delete this instance first.

Obj 2, Instance UID ['1.3.12.2.1107.5.2.33.37105.2015011615583122750227944']
Failure reason: A DICOM instance with the same StudyInstanceUID, SeriesInstanceUID, and SopInstanceUID has already been stored. If you wish to update the contents, delete this instance first.

Obj 3, Instance UID ['1.3.12.2.1107.5.2.33.37105.2015011615583076829427916']
Failure reason: A DICOM instance with the s

# Deletion

Delete study

In [87]:
study_uid = "1.2.840.113619.2.81.290.1.33656.20140218.202536"

url = f'{base_url}/studies/{study_uid}'
response = client.delete(url, headers=headers) 

In [88]:
print(f"Status: {response.status_code}\n")
print(response.content)

Status: 204

b''
